<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span><ul class="toc-item"><li><span><a href="#Стандартизируем-тестовую-и-валидную-выборку" data-toc-modified-id="Стандартизируем-тестовую-и-валидную-выборку-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Стандартизируем тестовую и валидную выборку</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span><ul class="toc-item"><li><span><a href="#Дерево-решения" data-toc-modified-id="Дерево-решения-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Дерево решения</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [2]:
data = pd.read_csv('/datasets/Churn.csv')

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Удалим первые 3 столбца RowNumber, CustomerId, Surname как бесполезные

In [4]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           9091 non-null   float64
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


In [6]:
data['Tenure'].describe()

count    9091.000000
mean        4.997690
std         2.894723
min         0.000000
25%         2.000000
50%         5.000000
75%         7.000000
max        10.000000
Name: Tenure, dtype: float64

Видим что есть пропуски в столбце Tenure, т.к. среднее и медиана имеет практически одинаковое значение, заменим пропуски на 5

In [7]:
data['Tenure']=data['Tenure'].fillna(5).astype('int64')

In [8]:
data['Tenure'].describe()

count    10000.00000
mean         4.99790
std          2.76001
min          0.00000
25%          3.00000
50%          5.00000
75%          7.00000
max         10.00000
Name: Tenure, dtype: float64

Проверим на дубликаты

In [9]:
data.duplicated().sum()

0

В столбцах Geography и Gender преобразуем категориальные признаки в численные методом прямого кодирования

In [10]:
data_dummies = pd.get_dummies(data, drop_first=True)
data_dummies.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


Сделаем выборки:

Обучающая <b>features_train, target_train</b> Валидационная <b>features_valid, target_valid</b> Тестовая <b>features_test, target_test</b>

In [11]:
target=data_dummies['Exited']
features=data_dummies.drop(['Exited'],axis=1)

In [12]:
features_train, features_valid, target_train,target_valid = train_test_split(features, target, test_size=0.5, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=0.25, random_state=12345)

## Исследование задачи

### Стандартизируем тестовую и валидную выборку

У признаков наблюдаем разный масштаб, стандартизируем их.

In [13]:
numeric=['CreditScore','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember']
scaler=StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric]=scaler.transform(features_train[numeric])
features_valid[numeric]=scaler.transform(features_valid[numeric])
features_test[numeric]=scaler.transform(features_test[numeric])

/tmp/ipykernel_313/2397498521.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train[numeric]=scaler.transform(features_train[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


### Дерево решений

Обучим модель с помощью Дерева решенй, без учета баланса

In [14]:
dtc_f1 = 0
dtc_f1_depth = 0
for depth in range(1,10,1):
    model = DecisionTreeClassifier(max_depth=depth,random_state=1234)
    model.fit(features_train, target_train)
    prediction = model.predict(features_valid)
    f1 = f1_score(target_valid, prediction)
    if f1 > dtc_f1:
        dtc_f1 = f1
        dtc_f1_depth = depth
print('max_depth:', dtc_f1_depth, 'F1:', dtc_f1)

max_depth: 7 F1: 0.5495011511895626


In [15]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
dtc_auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC:', round(dtc_auc_roc, 4))

AUC-ROC: 0.7706


### Случайный лес

Обучим модель с помощью Случайного леса, без учета баланса

In [16]:
rfc_f1 = 0
rfc_f1_depth = 0

for max_depth in range(1,20,1):
    model = RandomForestClassifier(max_depth=max_depth, n_estimators=50, random_state=1234)
    model.fit(features_train, target_train)
    prediction = model.predict(features_valid)
    f1 = f1_score(target_valid, prediction)
    if f1 > rfc_f1:
        rfc_f1 = f1
        rfc_f1_depth = max_depth
print('max_depth:', rfc_f1_depth,'F1:', rfc_f1)

max_depth: 13 F1: 0.5712


In [17]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
rfc_auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC:', rfc_auc_roc)

AUC-ROC: 0.8355608566007049


### Логистическая регрессия

Обучим модель с помощью Логистической регрессии, без учета баланса

In [18]:
model = LogisticRegression()
model.fit(features_train, target_train)
prediction = model.predict(features_valid)

lr_f1 = f1_score(target_valid, prediction)

print('F1:', lr_f1)

F1: 0.39036144578313253


In [19]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

lr_auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('AUC-ROC:', lr_auc_roc)

AUC-ROC: 0.7403315803740851


In [20]:
final = {'Metrics': ['F1', 'AUC-ROC'],
 'DecisionTreeClassifier': [round(dtc_f1, 4), round(dtc_auc_roc, 4)],
 'RandomForestClassifier': [round(rfc_f1, 4), round(rfc_auc_roc,4) ],
 'LogisticRegression': [round(lr_f1, 4), round(lr_auc_roc, 4)]}
df_final = pd.DataFrame(final)

df_final

,Metrics,DecisionTreeClassifier,RandomForestClassifier,LogisticRegression
0,F1,0.5495,0.5712,0.3904
1,AUC-ROC,0.7706,0.8356,0.7403


Видим что лучший показатель у модели Случайный Лес с параметром max_depth: 13 F1: 0.5712, AUC-ROC: 0.8356

## Борьба с дисбалансом

Посмотрим на соотношение тех кто ушле и тех кто остался

In [21]:
target_train.value_counts()

0    3984
1    1016
Name: Exited, dtype: int64

Видим что соотношение 1:4, это говорит о том что есть дисбаланс в данных.

In [22]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

Так же для борьбы с дисбалансом можно привести количество оставшихся клиентов к количеству ушедших. Применим функцию downsample. Уменьшим кол-в пооложительных ответов в 4 раза

In [23]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

## Тестирование модели

### Дерево решения

Проверим модель при увеличении выборки

In [24]:
dtc_f1_up = 0
dtc_f1_depth_up = 0

for depth in range(1,10,1):
    model_dtc = DecisionTreeClassifier(max_depth=depth, random_state=1234)
    model_dtc.fit(features_upsampled, target_upsampled)
    prediction = model_dtc.predict(features_valid)
    f1 = f1_score(target_valid, prediction)
    if f1 > dtc_f1_up:
        dtc_f1_up = f1
        dtc_f1_depth_up = depth
print('max_depth:', dtc_f1_depth_up, 'F1:', dtc_f1_up)

max_depth: 6 F1: 0.5818181818181818


In [25]:
probabilities_valid = model_dtc.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

dtc_auc_roc_up = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC:', dtc_auc_roc_up)

AUC-ROC: 0.7634999186771483


Проверим модель при уменьшении выборки

In [26]:
dtc_f1_down = 0
dtc_f1_depth_down = 0

for depth in range(1,10,1):
    model=DecisionTreeClassifier(max_depth=depth, random_state=1234)
    model.fit(features_downsampled, target_downsampled)
    prediction = model.predict(features_valid)
    f1 = f1_score(target_valid, prediction)
    if f1 > dtc_f1:
        dtc_f1_down = f1
        dtc_f1_depth_down = depth
print('max_depth:', dtc_f1_depth_down, 'F1:', dtc_f1_down)

max_depth: 6 F1: 0.5554465161923453


In [27]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

dtc_auc_roc_down = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC:', dtc_auc_roc_down)

AUC-ROC: 0.7419589048522635


<b>Вывод:</b> на увеличенной выборке модель показала себя лучше, F1 вырос с 0.5495 до 0.5818

AUC-ROC снизилась с 0.7706 до 0.7634

### Случайный лес

Проверим модель при увеличении выборки

In [28]:
rfc_f1_up = 0
rfc_f1_depth_up = 0

for max_depth in range(1,20,1):
    model_rfc = RandomForestClassifier(max_depth=max_depth, n_estimators=50, random_state=1234)
    model_rfc.fit(features_upsampled, target_upsampled)
    prediction = model_rfc.predict(features_valid)
    
    f1 = f1_score(target_valid, prediction)
    if f1 > rfc_f1_up:
        rfc_f1_up = f1
        rfc_f1_depth_up = max_depth
print('max_depth:', rfc_f1_depth_up,'F1:', rfc_f1_up)

max_depth: 11 F1: 0.6024242424242424


In [29]:
probabilities_valid = model_rfc.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

rfc_auc_roc_up = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC:', rfc_auc_roc_up)

AUC-ROC: 0.8291396042287883


Проверим модель при уменьшении выборки

In [30]:
rfc_f1_down = 0
rfc_f1_depth_down = 0

for max_depth in range(1,20,1):
    model = RandomForestClassifier(max_depth=max_depth, n_estimators=50, random_state=1234)
    model.fit(features_downsampled, target_downsampled)
    prediction = model.predict(features_valid)
    
    f1 = f1_score(target_valid, prediction)
    if f1 > rfc_f1_down:
        rfc_f1_down = f1
        rfc_f1_depth_down = max_depth
print('max_depth:', rfc_f1_depth_down,'F1:', rfc_f1_down)

max_depth: 6 F1: 0.5779583544946674


In [31]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

rfc_auc_roc_down = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC:', rfc_auc_roc_down)

AUC-ROC: 0.8285881268636487


<b>Вывод:</b> на увеличенной выборке модель показала себя лучше, F1 вырос с 0.5712 до 0.6024

AUC-ROC снизился с 0.8355 до 0.8291

### Логистическая регрессия

Проверим модель при увеличении выборки

In [32]:
model_lr = LogisticRegression(solver='liblinear')
model_lr.fit(features_upsampled, target_upsampled)
prediction = model_lr.predict(features_valid)

lr_f1_up = f1_score(target_valid, prediction)

print('F1:', lr_f1_up)

F1: 0.4782214156079854


In [33]:
probabilities_valid = model_lr.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

lr_auc_roc_up = roc_auc_score(target_valid, probabilities_one_valid)

print('AUC-ROC:', lr_auc_roc_up)

AUC-ROC: 0.7534421252371917


Проверим модель при уменьшении выборки

In [34]:
model = LogisticRegression(solver='liblinear')
model.fit(features_downsampled, target_downsampled)
prediction = model.predict(features_valid)

lr_f1_down = f1_score(target_valid, prediction)

print('F1:', lr_f1_down)

F1: 0.471623405191377


In [35]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

lr_auc_roc_down = roc_auc_score(target_valid, probabilities_one_valid)

print('AUC-ROC:', lr_auc_roc_down)

AUC-ROC: 0.7461417186229331


На увеличенной выборке модель показала себя лучше, F1 вырос с 0.3903 до 0.4782

AUC-ROC повысился с 0.7403 до 0.7534

In [36]:
final = {'Metrics': ['F1_up', 'F1_down', 'AUC-ROC_up', 'AUC-ROC_down'],
 'DecisionTreeClassifier': [round(dtc_f1_up, 4), round(dtc_f1_down, 4), round(dtc_auc_roc_up, 4), round(dtc_auc_roc_down, 4)],
 'RandomForestClassifier': [round(rfc_f1_up, 4), round(rfc_f1_down,4), round(rfc_auc_roc_up,4), round(rfc_auc_roc_down,4)],
 'LogisticRegression': [round(lr_f1_up, 4), round(lr_f1_down, 4), round(lr_auc_roc_up, 4), round(lr_auc_roc_down, 4)]}
df_final = df_final.append(pd.DataFrame(final))

df_final

,Metrics,DecisionTreeClassifier,RandomForestClassifier,LogisticRegression
0,F1,0.5495,0.5712,0.3904
1,AUC-ROC,0.7706,0.8356,0.7403
0,F1_up,0.5818,0.6024,0.4782
1,F1_down,0.5554,0.5780,0.4716
2,AUC-ROC_up,0.7635,0.8291,0.7534
3,AUC-ROC_down,0.7420,0.8286,0.7461


<b> Вывод: Лучше всего себя показала модель RandomForestClassifier на увеличенной выборке с показателем F1 = 0.6024 и AUC-ROC = 0.8291. В целом борьба с балансом показала себя с улучшением показателей по сравнению с несбалансированными данными.

In [37]:
model_rfc = RandomForestClassifier(max_depth=11, n_estimators=50, random_state=1234)
model_rfc.fit(features_upsampled, target_upsampled)
prediction = model_rfc.predict(features_test)
f1 = f1_score(target_test, prediction)
print('max_depth:', 11,'F1:', f1)

max_depth: 11 F1: 0.605009633911368


In [38]:
probabilities_test = model_rfc.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]

rfc_auc_roc_up = roc_auc_score(target_test, probabilities_one_test)
print('AUC-ROC:', rfc_auc_roc_up)

AUC-ROC: 0.8647402260875198


<b> Вывод: Выбранная нами модель RandomForestClassifier на тестовой выборке показала себя еще лучше и показала следующие показатели F1 = 0.6050 и AUC-ROC = 0.8647.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: задача исследована
    - [ ]  Исследован баланс классов
    - [ ]  Изучены модели без учёта дисбаланса
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 3: учтён дисбаланс
    - [ ]  Применено несколько способов борьбы с дисбалансом
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 4: проведено тестирование
- [ ]  Удалось достичь *F1*-меры не менее 0.59
- [ ]  Исследована метрика *AUC-ROC*